In [52]:
data = open('recodata/u.links')
Links = {}
i = 0
lines = data.read().split('\n')
for line in lines[:-2]:
    TabLine = line.split('\t')
    Id,Successors = TabLine[0],TabLine[1:]
    Links[int(Id)]=map(int,Successors)
    i = i+1
Links

{1: [520, 510, 876, 107, 800, 702, 691, 252],
 2: [45,
  89,
  24,
  909,
  350,
  252,
  330,
  473,
  694,
  310,
  530,
  258,
  830,
  218,
  339,
  658,
  615,
  419,
  76,
  12,
  221,
  684,
  267,
  444,
  664,
  368,
  842,
  60,
  409],
 3: [452, 475, 554, 576, 545, 867, 209, 609],
 4: [66, 377, 432, 511, 127, 886, 436, 856, 173, 261, 593],
 5: [584, 245, 797, 369, 413, 612, 438, 614],
 6: [444, 512, 853, 469, 502, 791],
 7: [78, 909, 36, 696, 598, 138, 767, 61],
 8: [274,
  395,
  298,
  553,
  433,
  510,
  598,
  797,
  138,
  259,
  556,
  513,
  118,
  613,
  910,
  515,
  835,
  879,
  836,
  717,
  481,
  164,
  340,
  166,
  122,
  365,
  421,
  125,
  169,
  345,
  422,
  785,
  369,
  468,
  800,
  647,
  702,
  626,
  725,
  528,
  903,
  629],
 9: [291,
  270,
  294,
  195,
  174,
  350,
  394,
  472,
  330,
  276,
  157,
  311,
  312,
  697,
  776,
  634,
  711,
  536,
  14,
  680,
  287,
  540,
  322,
  763,
  840,
  621,
  303,
  402,
  645,
  60,
  109,
  506,

In [9]:
import pandas as pd

In [53]:
len(Links.keys())

933

In [4]:
import numpy as np
import scipy.sparse as sp
import pandas as pd

def get_sparse_matrix_from_dic_dense(dic):
    l=0
    for user_id, movie_ids in dic.items():
        for movie_id in movie_ids:
            l=max(l,movie_id,user_id)
    mat = sp.dok_matrix((l,l), dtype=np.int8)
    for user_id, movie_ids in dic.items():
        for movie_id in movie_ids:
            mat[user_id-1,movie_id-1]=1

    return mat.todense()


In [51]:
def loss(R,L,P,Q,Z,lamb):
    ''' R : matrix_ratings
        L : matrix_links
        P : matrix_item_feature
        Q : matrix_user_feature
        Z : matrix_links_feature'''
    C1 = get_mask_rating(R)
    C2 = get_mask_rating(L)
    PQ = P.T.dot(Q)

    ratings = R
    PZ = Z.T.dot(Q)
    links = L
    resRating_PQ = np.sum(np.square(np.nan_to_num((ratings-PQ)*C1)))
    resLinks_PZ = np.sum(np.square(np.nan_to_num((links-PZ)*C2)))
    res2 = np.trace(P.dot(P.T))**2 + np.trace(Q.dot(Q.T))**2 + np.trace(Z.dot(Z.T))**2 
    loss = resRating_PQ + resLinks_PZ + lamb*res2
    return loss

def get_mask_rating(R):
    Mask = np.where(R>0,1,0)
    return Mask

def error(R,P,Q,idx1,idx2):
    return R[idx_item,idx_user] - (P[:,idx_user].T.dot(Q[:,idx_item]))
    
def descente_de_gradient(It,pas,R,L,P,Q,Z,lamb):
    nbrItem , nbrUser = R.shape
    idx_matrix_ratings = np.where(np.isfinite(R))
    nombre_index = idx_matrix_ratings[0].shape[0]
    idx_matrix_links = np.where
    Loss = {}
    i=0
    while i < It:
        #get idx (item,user,factor_user = link)
        
        idx_ratings = np.random.randint(0, nombre_index-1, 1)
        idx_user = idx_matrix_ratings[1][idx_ratings]
        idx_item = idx_matrix_ratings[0][idx_ratings]
        idx_matrix_links = np.where(L[idx_user,:]>0)
        linked = idx_matrix_links[1]
        nbr_link = linked.shape[0]
        if (nbr_link>1):
            idx_link = np.random.randint(0,nbr_link-1,1)
        else:
            idx_link = 0
        idx_link = linked[idx_link]
        
        #get error for ratings and for User links
        roU_I = R[idx_item,idx_user] - (Q[:,idx_user].T.dot(P[:,idx_item]))
        roU_L = L[idx_user,idx_link] - (Q[:,idx_user].T.dot(Z[:,idx_link]))
        
        print (1-lamb*pas)*Q[:,idx_user]
        #mise à jour de P(user),Q(user),L(user)      
        P[:,idx_item] = (1-lamb*pas)*P[:,idx_item]- pas*Q[:,idx_user]*roU_I
        
        Q[:,idx_user] = (1-lamb*pas)*Q[:,idx_user]- pas*P[:,idx_item]*roU_I-pas*Z[:,idx_link]*roU_L
        
        Z[:,idx_link] = (1-lamb*pas)*Z[:,idx_link]- pas*Q[:,idx_user]*roU_L
        
        #calcul loss
        Loss[i] = loss(R,L,P,Q,Z,lamb)
        print Loss[i]
        i += 1
    #plot loss
    plt.figure()
    plt.plot(Loss)
    plt.show()
    Matrix_rating = dot(P.T,Q)
    return Loss,Matrix_rating

In [52]:
A = np.array([[1,2,3],[1,2,3]])
B = np.array([[1,2,3],[1,2,5]])
A*B

array([[ 1,  4,  9],
       [ 1,  4, 15]])

In [53]:
def loadMovieLensLink():
    data = open('recodata/u.links')
    Links = {}
    i = 0
    lines = data.read().split('\n')
    for line in lines[:-2]:
        TabLine = line.split('\t')
        Id,Successors = TabLine[0],TabLine[1:]
        Links[int(Id)]=map(int,Successors)
        i = i+1
    return Links

def loadMovieLens():

  
  # Load data
  prefs={}
  for line in open('recodata/u.data'):
    (user,movieid,rating,ts)=line.split('\t')
    prefs.setdefault(user,{})
    prefs[user][movieid]=float(rating)
  return prefs


In [54]:
Links = loadMovieLensLink()
L = get_sparse_matrix_from_dic_dense(Links)

In [55]:
Ratings = loadMovieLens()
R = pd.DataFrame(Ratings)
R = R.values

In [56]:
def initialize_Parameters(R,sizeFeature):
    numberOfItem, numberOfUser = R.shape
    P = np.random.randn(sizeFeature,numberOfItem)
    Q = np.random.randn(sizeFeature,numberOfUser)
    Z = np.random.randn(sizeFeature,numberOfUser)
    return P,Q,Z

In [57]:
P,Q,Z = initialize_Parameters(R,3)

In [58]:
descente_de_gradient(It=1000,lamb=0,pas=0.001,R=R,L=L,P=P,Q=Q,Z=Z)

[[ 0.17995019]
 [-0.50036368]
 [-0.47192521]]
34740384.9289
[[-0.71621056]
 [-1.74651734]
 [-0.03366588]]
34740722.1823
[[ 1.44166913]
 [-1.23574583]
 [-0.93263827]]
34740779.3812
[[ 1.89045629]
 [ 0.29357811]
 [ 0.70095761]]
34740850.6844
[[ 2.80754359]
 [-1.12950648]
 [-0.59798106]]
34741311.951
[[-0.25075222]
 [ 0.11407683]
 [-1.74219602]]
34741118.2538


IndexError: index 0 is out of bounds for axis 0 with size 0